<a href="https://colab.research.google.com/github/uakarsh/latr/blob/main/examples/textvqa/LaTr_TextVQA_Part_1_Getting_to_know_the_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Downloading the requirements

!pip install -q transformers
!pip install -q sentencepiece==0.1.91
!pip install -q pytesseract
!sudo apt install -q tesseract-ocr
!pip install -q 'Pillow==7.1.2'

     |████████████████████████████████| 4.2 MB 8.4 MB/s 
     |████████████████████████████████| 596 kB 65.0 MB/s 
     |████████████████████████████████| 6.6 MB 58.3 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 1.1 MB 8.4 MB/s 
     |████████████████████████████████| 3.1 MB 9.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesse

In [ ]:
!git clone https://github.com/uakarsh/latr.git
import sys
sys.path.append('/content/latr/src/latr/')

Cloning into 'latr'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 132 (delta 46), reused 80 (delta 20), pack-reused 0
Receiving objects: 100% (132/132), 154.03 KiB | 1.47 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [ ]:
## Default Library import

import os
import json
import numpy as np
from tqdm.auto import tqdm
import pandas as pd
import json

## For the purpose of displaying the progress of map function
tqdm.pandas()


## Visualization libraries
from google.colab.patches import cv2_imshow
import pytesseract
from PIL import Image, ImageDraw

## Specific libraries of LaTr
import torch.nn as nn
from dataset import load_json_file, get_specific_file, resize_align_bbox, get_tokens_with_boxes, create_features
from modeling import LaTr_for_pretraining

## Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
## Setting the hyperparameters as well as primary configurations

PAD_TOKEN_BOX = [0, 0, 0, 0]
max_seq_len = 512
batch_size = 1
t5_model = "t5-base"

In [ ]:
## The TextVQA Dataset is present in my drive
## It can be downloaded from this link: https://drive.google.com/drive/folders/1bR03cRGnPENCiUaurKSGTSNQHUQlsDCR?usp=sharing and place it in the google drive, thats it!!

base_path = 'drive/MyDrive/TextVQA/'
ocr_json_path = os.path.join(base_path, 'TextVQA_Rosetta_OCR_v0.2_train.json')
train_json_path = os.path.join(base_path, 'TextVQA_0.5.1_train.json')
train_images = os.listdir(os.path.join('drive/MyDrive/TextVQA', 'train_images'))
train_images = pd.DataFrame({'images': train_images})
train_images['images'] = train_images['images'].progress_apply(lambda x: os.path.join(os.path.join('drive/MyDrive/TextVQA', 'train_images'), x))

## Converting to list
train_images = train_images['images'].tolist()

  0%|          | 0/19310 [00:00<?, ?it/s]

In [ ]:
train_ocr_json = json.load(open(ocr_json_path))['data']
train_json = json.load(open(train_json_path))['data']

In [ ]:
## Useful for the key-value extraction

train_json_df = pd.DataFrame(train_json)
train_ocr_json_df = pd.DataFrame(train_ocr_json)

In [ ]:
train_json_df.drop(columns = ['flickr_original_url', 'flickr_300k_url',
                              ], axis = 1, inplace = True)

In [ ]:
## Deleting the json

del train_json
del train_ocr_json

In [ ]:
grouped_df = train_json_df.groupby('image_id')

In [ ]:
grouped_df.groups.keys()